In [1]:
from yahoo_earnings_calendar import YahooEarningsCalendar
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import pandas_datareader as pdr
import dateutil.parser
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
%%html
<!-- Stop Line wrap inside the DF -->

<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [3]:
def get_earnings_data(list_of_tickers):

    """
    Get the historical earnings data for a list of tickers
    """

    start_date = datetime.now().date() - timedelta(720)
    end_date = datetime.now().date() + timedelta(30)

    earnings_df = pd.DataFrame()

    for ticker in list_of_tickers:
        # Downloading the earnings calendar for each ticker
        earnings_calendar = YahooEarningsCalendar()
        earnings_info = earnings_calendar.get_earnings_of(ticker)
        earnings_df = earnings_df.append(earnings_info)

    # Extracting the date from the string and filtering for the period of interest
    earnings_df['report_date'] = earnings_df['startdatetime'].apply(lambda x: dateutil.parser.isoparse(x).date())
    earnings_df = earnings_df.loc[earnings_df['report_date'].between(start_date, end_date)]
    
    # Update the column names to follow PEP8
    earnings_df = earnings_df.rename(columns={'epsestimate': 'eps_estimate', 'epsactual': 'eps_actual',
                                              'epssurprisepct': 'eps_surprise_pct'})
    # Only use the columns we need
    earnings_df = earnings_df[['ticker', 'eps_estimate', 'eps_actual', 'eps_surprise_pct', 'report_date']]

    # Drop nan rows if we do not have an eps_estimate 
    # - The API provided a report_date for uber of 2/15/19 but their first pubic earnings was 5/30/19 
    # - Drop the row since we will use this column later on in our probability analysis 
    # - Also saves on storage since the EOD API calls will return less data
    earnings_df = earnings_df[pd.notnull(earnings_df['eps_estimate'])]
    
    return earnings_df

In [4]:
def get_eod_data(x):

    """
    Get the EOD data for the tickers in our earnings DF
    """

    ticker = x['ticker'].iloc[0]

    # Get the start/end date from our earnings DF to determine the range of EOD data we need    
    #   - Minus one day from start_date to compute the % change in price for the reporting_date
    #   - Add one day from end_date to compute the % change in price for post earnings reactions
    start_date = x['report_date'].iloc[-1]
    start_date = start_date - BDay(1)
    end_date = x['report_date'].iloc[0]
    end_date = end_date + BDay(1)

    results = pdr.DataReader(ticker, 'yahoo', start_date, end_date)

    # Update the column names to follow PEP8
    results = results.rename(columns={'High': 'high', 'Low': 'low', 'Open': 'open', 'Close': 'close',
                                      'Volume': 'vol', 'Adj Close': 'adj_close'})
    
    return results

In [5]:
def compute_eod_analytics(eod_df):

    """
    Calculate the percent change in price and update data for matching process
    """

    # Calculate the % change in the equity price
    eod_df['previous_price'] = eod_df.groupby(['ticker'])['close'].transform(lambda x: x.shift())
    eod_df['daily_prc_change'] = (((eod_df['close'] - eod_df['previous_price']) / 
                                   eod_df['previous_price']) * 100).round(2)

    # Shift backwards b/c we want ot join on the reporting_date (pre-results) but view the post earnings EOD data
    eod_df['reaction_daily_prc_chang'] = eod_df.groupby(['ticker'])['daily_prc_change'].transform(lambda x: x.shift(-1))    
    eod_df['open'] = eod_df.groupby(['ticker'])['open'].transform(lambda x: x.shift(-1))
    eod_df['close'] = eod_df.groupby(['ticker'])['close'].transform(lambda x: x.shift(-1))
    eod_df['high'] = eod_df.groupby(['ticker'])['high'].transform(lambda x: x.shift(-1))
    eod_df['low'] = eod_df.groupby(['ticker'])['low'].transform(lambda x: x.shift(-1))

    # Filer for only columns we want
    eod_df = eod_df[['ticker', 'trade_date', 'daily_prc_change', 'reaction_daily_prc_chang', 'open', 'high', 'low', 'close']]

    return eod_df

In [6]:
def join_earnings_and_eod_data(earnings_df, eod_df):

    """
    Join the earnings and eod data sets
    """
    # Update columns to the same data type
    earnings_df['report_date'] = pd.to_datetime(earnings_df['report_date'])
    eod_df['trade_date'] = pd.to_datetime(eod_df['trade_date'])
    
    # Left join b/c we only want matching EOD where we have earnings data
    df = pd.merge(earnings_df, eod_df, how='left', left_on=['report_date', 'ticker'], right_on=['trade_date', 'ticker'])
    
    # Roll the trade_date forward to align with the EOD numbers
    df['trade_date'] = df['trade_date'] + BDay(1)
    
    # Drop un-used columns
    df = df.drop(['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y'], errors='ignore', axis=1)

    return df

In [17]:
if __name__ == '__main__':
    
    # Get market data via Yahoo earnings & market data API's
    tickers = ['AAPL', 'TSLA', 'FB', 'UBER']
    earnings_data = get_earnings_data(tickers)
    eod_data = earnings_data.groupby(['ticker']).apply(lambda x: get_eod_data(x)).reset_index()    
    
    # rename after the reset_index since Date was an index
    eod_data = eod_data.rename(columns={'Date': 'trade_date'})
    
    # Update EOD data and prep for matching
    eod_data = compute_eod_analytics(eod_data)

    # Join data sets and review results
    earnings_and_eod_data = join_earnings_and_eod_data(earnings_data, eod_data)

In [20]:
earnings_and_eod_data.to_csv('data/earnings_and_eod_data_backup.csv')

In [18]:
earnings_and_eod_data.loc[earnings_and_eod_data['ticker'] == 'UBER']

,ticker,eps_estimate,eps_actual,eps_surprise_pct,report_date,trade_date,daily_prc_change,reaction_daily_prc_chang,open,high,low,close
24,UBER,-0.65,NaN,NaN,2020-11-05,NaT,NaN,NaN,NaN,NaN,NaN,NaN
25,UBER,-0.86,-1.02,-18.06,2020-08-06,2020-08-07,4.55,-5.21,33.090000,33.700001,32.450001,32.900002
26,UBER,-0.88,-1.70,-93.62,2020-05-07,2020-05-08,11.18,6.01,32.590000,33.299999,31.639999,32.790001
27,UBER,-0.67,-0.64,5.19,2020-02-06,2020-02-07,0.76,9.54,39.980000,41.000000,38.860001,40.630001
28,UBER,-0.81,-0.68,16.56,2019-11-04,2019-11-05,-0.92,-9.85,29.129999,29.299999,27.969999,28.020000
29,UBER,-3.19,-4.72,-47.82,2019-08-08,2019-08-09,8.24,-6.80,39.560001,40.680000,38.709999,40.049999
30,UBER,-2.26,-2.23,1.33,2019-05-30,2019-05-31,-0.35,1.53,41.150002,41.570000,39.410000,40.410000
